### Notebook for visualization ground truth dataset

For now, organized with ethograms in dictionary, and videos saved in `/data/caitlin/groud_truth`

In [31]:
import pandas as pd
from ipywidgets import HBox, VBox, Select
from fastplotlib import ImageWidget, Plot
from mesmerize_core.arrays import LazyVideo
from pathlib import Path
import numpy as np
from fastplotlib.graphics.selectors import LinearSelector
from fastplotlib.graphics import ImageGraphic

In [32]:
ETHOGRAM_COLORS = {
    "lift": "b",
    "handopen": "green",
    "grab": "r",
    "sup": "cyan",
    "atmouth": "magenta",
    "chew": "yellow"
}

In [33]:
df = pd.read_hdf("/data/caitlin/ground_truth.hdf")

In [34]:
options = [k for k in list(df["trial"])]

In [35]:
len(options)

291

In [36]:
data_path = Path('/data/caitlin/ground_truth/')

In [37]:
def trial_change(obj):
    selected_vid = data_path.joinpath(trial_selector.value).with_suffix('.avi')
    new_data = [LazyVideo(selected_vid)]
    # iw.gridplot[0,0].clear()
    # iw.gridplot[0,0].add_graphic(ImageGraphic(new_data[0][0]))
    iw.set_data(new_data=new_data, reset_vmin_vmax=True)
    plot.clear()
    make_plot(plot)
    plot.auto_scale()

In [38]:
trial_selector = Select(options=options)
trial_selector.observe(trial_change, "value")

In [39]:
iw = ImageWidget(data=[LazyVideo(data_path.joinpath(trial_selector.value).with_suffix('.avi'))])

RFBOutputContext()

/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:87: UserWarning: min not implemented for LazyTiff, returning min of 0th index
  warn("min not implemented for LazyTiff, returning min of 0th index")
/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:92: UserWarning: max not implemented for LazyTiff, returning min of 0th index
  warn("max not implemented for LazyTiff, returning min of 0th index")


In [40]:
def ethogram_event_handler(ev):
    ix = ev.pick_info["selected_index"]
    iw.sliders["t"].value = ix

In [41]:
plot = Plot(size=(500, 100))
def make_plot(plot):
        
    ix = df.loc[df["trial"] == trial_selector.value].index[0]
    ethogram_array = df.loc[:,'ethogram'].loc[ix] 
    y_bottom = 0
    for i, b in enumerate(ETHOGRAM_COLORS.keys()):
        xs = np.arange(ethogram_array.shape[1], dtype=np.float32)
        ys = np.zeros(xs.size, dtype=np.float32)

        lg = plot.add_line(
            data=np.column_stack([xs, ys]),
            thickness=10,
            name=b
        )

        lg.colors = 0
        lg.colors[ethogram_array[i] == 1] = ETHOGRAM_COLORS[b]

        y_pos = (i * -10) - 1
        lg.position_y = y_pos

    ethogram_selector = LinearSelector(
                selection=0,
                limits=(0, ethogram_array.shape[1]),
                axis="x",
                parent=lg,
                end_points=(y_bottom, y_pos),
            )

    plot.add_graphic(ethogram_selector)
    ethogram_selector.selection.add_event_handler(ethogram_event_handler)

RFBOutputContext()

In [42]:
make_plot(plot)
VBox([
    HBox([iw.show(), trial_selector]),
    plot.show()
])